In [1]:
#### create a jet
"""
Basic docstring explaining example
"""
from __future__ import print_function
import sys
import os
#change this path for the correct ones in your computer.

myhost = os.uname()[1]
if myhost == 'jesus-Latitude-3310':
    print ('you are in the lap')
    # Import sf3dmodels 
    sys.path.append('/home/jesus/star-forming-regions')
    #import radmc3Py
    sys.path.append('/home/jesus/bin/python/radmc3dPy/')
    #import RRL csv file properties 
    lines_properties_file = "/utils/Hydrogen_recom_lines_in_use.csv"
    #import RADMC3D 0.4 for RRL
    radmc_path = "/home/jesus/radmc-3d/version_0.40/examples/run_recomblines_userdef"

elif myhost == 'posgrado30.crya.privado':
    print ('you are in the office PC ')
    sys.path.append('/fs/posgrado30/other0/jesus/radmc3dPy/lib/python3.9/site-packages')
    # Import sf3dmodels
    sys.path.append('/fs/posgrado30/other0/opt/star-forming-regions')
    #import RADMC for RRL
    radmc_path = "/fs/posgrado30/other0/jesus/radmc-3d/version_0.40/examples/run_recomblines_userdef"
    # import csv file with RRL properties
    lines_properties_file = "../../utils/Hydrogen_recom_lines_in_use.csv.csv"
    # import ngVLA calculator
    sys.path.append("../../ngvla_sensitivity_calculator_master")
    

elif myhost == '':
    print ('you are in the ---- server')
    #sys.path.append('/share/Part1/jesus/SF_regions/utils')
else:
    print ('the packages path is not defined')

#********************
#sf3dmodels libraries
#********************
import sf3dmodels.Plot_model as Pm            #Plotting model
from sf3dmodels.outflow import OutflowModel   #Model functions
import sf3dmodels.utils.units as u            #Units
import sf3dmodels.rt as rt                    #Writing functions for radiative transfer
import sf3dmodels.Model as Model              #Grid
from sf3dmodels.grid import Overlap           #Overlap submodels
from radmc3dPy.image import *
import radmc3dPy.image as image
from matplotlib import cm
from matplotlib import pyplot as plt
#from plot_helpers import plot_1d_props#, _props_jets

### import ngVLA noise calculator
from ngVLA_sensitivity_calculator import *

##------
# import modules
#----------------
import astropy.wcs as wcs
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel
#from spectral_cube import SpectralCube
#from pvextractor import extract_pv_slice, Path, PathFromCenter
import radmc3dPy.image as image
#from plot_helpers import plot_1d_props, plot_1d_props_jaquez_dens_vs_radius
import matplotlib.pyplot as plt
#-----------------
#Extra libraries
#-----------------
import scipy.fft
from matplotlib import colors
import numpy as np
import os
import subprocess
import time
import astropy.units as asu
import astropy.constants as cts
import pandas as pd
os.getcwd()

you are in the office PC 


'/fs/posgrado30/other0/jesus/respaldo_paper2/ngVLA_SF_regions/jets/sky_models'

# Make the grid
# run sf3dmodels

In [ ]:
t0 = time.time()
#--------------------
### The resolution to the jets grid.
dx_grid = 2*u.au # 2.8 mas
#*********************
#OUTFLOW 1: standar collimated
#*********************
tag = '_outflow_collimated'
#---------------------
#GEOMETRIC PARAMETERS
#---------------------
pos_c = np.array([0, 0, 0]) #Origin of coordinates on the axis.
axis = np.array([0,0,1])    #Long axis direction, of arbitrary length. For example, an axis pointing to the z-direction: axis = [0,0,1] 
z_min = 25*u.au      #Lower and upper limits to compute the physical properties  # u.au = au to m convertion value
z_max = 150*u.au  #maximum value for the jet.
#---------------------
#PHYSICAL PARAMETERS
#---------------------
mdot = 1.5e-6 *u.MSun_yr #1.0e-7 *u.MSun_yr # 3 veces menos brilloso que el del paper == 1.5e-7*u.MSun_yr #0.1e-7    # transform to Msun/year to  a kg/s
mu = 1.6735e-27          # Kg ; masa H
cte = 1.0 #4.9975e-7    #esta constante no he descubierto de donde sale la voy a cancelar. 
#---------------------
#PHYSICAL PROPERTIES OF THE REYNOLDS 86 JET MODEL
#---------------------
r0=z_min               #inicio del jet
v0 = 500 * 1e3         #  Gas speed at `z0`. Speed normalization factor. 500 es km/s, pero convierto a metros porque el codigo trabaja en mks
#####
# The formula is w = w0 (r/r0)^\epsilon
#where w is the widht of the jet
#####
w0 = 5*u.au            # parameters to compute the Jet half-width at each z. poner en au y se transforma a m. 
eps = 2./3.            # eps = 1 == corresponds to a conical (constant opening angle) jet
w = [w0, eps]

s = np.pi*w0**(2)                                   # calculado con ec 12 paper. w0 en m
n =cte*mdot*mu**(-1)*v0**(-1)*s**(-1)               # en m-3 # Reynolds equaation 16.5
print('Densidad jet en la base = ', n, ' m-3')

# power law (?) for the temperature
T0 = 10000.
qT = 0
temp = [T0, qT]

dens0 = n #* 1e6 # paso la densidad a m-3 porque el codigo trabaja en mks
qn = -2*eps
dens = [dens0, qn]

ionfrac = [1, 0]           #all the gas is ionized

abund = [1e-4, 0]          # parameters to compute the molecular abundance
gtd = 100                  # gas to dust ratio
#---------------------
#COMPUTING MODEL PROPERTIES
#---------------------
Outf1 = OutflowModel(pos_c, axis, z_min, z_max, dx_grid) #Initializing Class with grid parameters
Outf1.reynolds86(w, dens, ionfrac, temp, v0, abund, gtd, [0,0,0], r0) #Invoking the outflow model from Reynolds et al. 1986

#---------------------
#WRITING FOR RADMC3D
#---------------------
#Using the Radmc3d class
prop1 = {'vel_x' : Outf1.vx, 'vel_y' : Outf1.vy, 'vel_z' : Outf1.vz,
         'dens_e': Outf1.density_ion,
         'dens_ion': Outf1.density_ion,
         'temp_gas': Outf1.temperature}
radmc1 = rt.Radmc3d(Outf1.GRID)
radmc1.submodel(prop1, output='datatab'+tag+'.dat')
print('Output columns', radmc1.columns)

#********* End collimated jet
#*******************************************************************

#*******************************************************************
#OUTFLOW 2: conical jet # to simulate winds?
#*********************
tag = '_outflow_conical'
#---------------------
#GEOMETRY
#---------------------
pos_c = np.array([0, 0, 0])   #[200*u.au, -50*u.au, 0]
axis = np.array([0,0,1])      #[1,1,-1]
z_min = 3*u.au
z_max = 150*u.au

#---------------------
#PHYSICAL PARAMETERS
#---------------------
#also put the same accretion rate
mdot = 1.75e-6 *u.MSun_yr #3.5e-7*u.MSun_yr # 3 veces menos brilloso que el del paper == 1.75e-7*u.MSun_yr #0.5e-7*u.MSun_yr  # convierte masa solar por año a kg/s
#---------------------
#PHYSICAL PROPERTIES
#---------------------
r0=z_min
v0 = 100 * 1e3 #100 es km/s, pero convierto a metros porque el codigo trabaja en mks
w0 = 5*u.au  #  poner en au. La unidad es m
eps = 1
w = [w0, eps]

T0 = 10000.
qT = -0.5    # esta cosa tiene una variacion de la temperatura con z/z0

temp = [T0, qT]

s = np.pi*w0**(2)   # calculado con ec 12 paper. w0 en m
n = cte*mdot*mu**(-1)*v0**(-1)*s**(-1)   # en m-3
print('Densidad viento en la base = ',n ,' m-3')

dens0 = n #* 1e6   # paso la densidad a m-3 porque el codigo trabaja en mks
qn = -2*eps   # -2
dens = [dens0, qn]

ionfrac = [0.1,-0.5]      #qx = -0.5 es como varia # 10% de ionizacion inicialmente

abund = [1e-4, 0]
gtd = 100

#---------------------
#COMPUTING MODEL
#---------------------
Outf2 = OutflowModel(pos_c, axis, z_min, z_max, dx_grid) #Initializing Class with grid parameters
Outf2.reynolds86(w, dens, ionfrac, temp, v0, abund, gtd, [0,0,0], r0) #Invoking the outflow model from Reynolds et al. 1986

#---------------------
#WRITING FOR RADMC3D
#---------------------
#Using the Radmc3d class
prop2 = {'vel_x' : Outf2.vx, 'vel_y' : Outf2.vy, 'vel_z' : Outf2.vz,
        'dens_e': Outf2.density_ion,
         'dens_ion': Outf2.density_ion,
         'temp_gas': Outf2.temperature}
radmc2 = rt.Radmc3d(Outf2.GRID)
radmc2.submodel(prop2, output='datatab'+tag+'.dat')
print('Output columns', radmc2.columns)

#####################################-------------------------
#JOIN the JETS 
#################################### -----------------------------------
#********
#GRIDDING both jets
#********
sizex = 150 * u.au       #500
sizey = sizez = 150 * u.au    #500
Nx = Ny = Nz = 150 # 2au the same that dx
GRID = Model.grid([sizex, sizey, sizez], [Nx, Ny, Nz], rt_code='radmc3d')

files = ['datatab_outflow_collimated.dat', 'datatab_outflow_conical.dat'] #

#### I add the columns for the vel_x, vel_y, vel_z in order to have the same columns that in the model disk
#### and to run recomblines instead that freefree in RADMC
columns = ['id', 'x', 'y', 'z', 'dens_ion', 'dens_e', 'temp_gas', 'vel_x', 'vel_y', 'vel_z']
outflows = Overlap(GRID)
finalprop = outflows.fromfiles(columns, submodels = files, rt_code = 'radmc3d')


#********************
#WRITING FOR RADMC-3D
#********************
print ("################ Writing for RADMC-3D ####################")
#read the lines properties to run RADMC
lines_df = pd.read_csv(lines_properties_file)
line = 3 # = H38a
line_prop = lines_df.iloc[line]
n_sup = line_prop["nsup"]
n_inf = line_prop["ninf"]
print (n_inf)
#line_freq = line_prop["Frequency_radmc"]
A = rt.Radmc3dDefaults(GRID)
A.recomblines(finalprop, [n_sup,n_inf], kwargs_control = {'userdef_nonlte': 1}) #Writing prop with radmc3d format, non-LTE
print ('ejecution time = {} minuts'.format((time.time()-t0)/60))

# Look for the luminosity at 5.8GHz (Purser 2017)

In [ ]:
#def Luminosity5_8GHZ:
# flujo a 5.8 GHz
########################################## RADIATIVE TRANSFER WITH RADMC-3D ####################################   
radmc_rl = os.path.join(radmc_path, 'radmc3d')
subprocess.run(radmc_rl+' image lambda 51688 incl {0} npix {1} sizeau {2} setthreads 8 '.format(60 ,npix,size_region), shell=True, executable='/bin/bash') 
os.system('mv image.out model_jet_continuum_5.8GHz.out')
distance = 2000
#### make where is set the model in the Purser plot
im=image.readImage('model_jet_continuum_5.8GHz.out')
a = image.plotImage(im, au=True, cmap=cm.gist_heat, bunit='snu', dpc=distance, ifreq=0)
Snu = a['implot'].get_array().sum()*asu.Jy
print (Snu)
SnuD2 = Snu* ((distance*asu.pc)**2)
print (SnuD2.to(asu.mJy * asu.kpc * asu.kpc))

# RUN RADMC FOR THE LINE OBSERVATIONS

In [ ]:
#-----------------------------------------
# This control the properties of the RADMC radiative transfer
#---------------------------------------
NPIX = npix = 2000             # cdelt = 0.41 mas/pix resolution ngVLA observation 2mas
CHANNEL_WIDTH =channel_width_vel = 30 #15      #km/s
NUMBER_CHANNELS = number_channels = 60 #80    #
INCLINATION= inclination = 60.0     #30   #
SIZE_REGION = size_region = 300*2       # au

#------------------------------------------
# This control the synthetic observation sky models creation
#------------------------------------------
#BEAM = beam = (4.7e-3,4.7e-3)      # arcsec
TIME_INT = time_int = 3600*5        #1hr # second
DISTANCE = distance = 2000        # pc
cdelt = (206265*asu.arcsec).to(asu.mas) * (size_region*asu.au) / ((distance*asu.pc).to(asu.au))
print (cdelt/npix)

In [ ]:
######## radio velocity to frequency definition
def freq2vel(freq, freq_0):
    return (freq_0 - freq) * (cts.c.cgs.to(asu.km/asu.s)) / freq_0
def vel2freq(vel,freq_0):
    return freq_0 * (1.-(vel/(cts.c.cgs.to(asu.km/asu.s))))
#read the lines properties to run RADMC
lines_df = pd.read_csv(lines_properties_file)
line = 3 #3=H41a #1=H38a
line_prop = lines_df.iloc[line]
n_sup = line_prop["nsup"]
n_inf = line_prop["ninf"]
line_freq = line_prop["Frequency_radmc"]
print (n_inf)
channel_width_velocity = channel_width_vel * asu.km/asu.s
channel_width_freq_0 = vel2freq(channel_width_velocity,line_freq*asu.Hz)
channel_width_freq = line_freq*asu.Hz - channel_width_freq_0 
print ("channel width {} or {}".format(channel_width_velocity, channel_width_freq.to(asu.MHz)))
# channel width from the ngLVA calculator = 4.6532191280142206 MHz
freq_ini = line_freq*asu.Hz - ((channel_width_freq)*(number_channels/2))
freq_fin = line_freq*asu.Hz + ((channel_width_freq)*(number_channels/2))
print ("{:.7e}, {:.7e}".format(freq_ini,freq_fin))
wave_ini = freq_fin.to(asu.micron, equivalencies=asu.spectral())
wave_fin = freq_ini.to(asu.micron, equivalencies=asu.spectral())
wave_range=[abs(wave_ini.value),abs(wave_fin.value)]
print ("the wave range is {} to {}".format(wave_ini,wave_fin))
#--------------------------------
i = inclination #30

In [ ]:
# LINE FULL DATA CUBE RT model
########################################## RADIATIVE TRANSFER WITH RADMC-3D ####################################   
radmc_rl = os.path.join(radmc_path, 'radmc3d')
##### JAQUEZ H38_\alpha -> 2600.6855 microns
subprocess.run(radmc_rl+' image lambdarange {} {} nlam {} incl {} npix {} sizeau {} setthreads 8 '.format(wave_range[0],wave_range[1],number_channels, inclination ,npix,size_region), shell=True, executable='/bin/bash') 
os.system('mv image.out papermodel_jet_line_H{}_60ch_30kms_incl60.out'.format(n_inf))

In [ ]:
im = image.readImage('papermodel_jet_line_H{}_60ch_30kms_incl60.out'.format(n_inf))
model = 'jet'
output = 'img_rl_{}_H{}_30kms_{}pc.fits'.format('jet',n_inf,int(distance))
im.writeFits(fname=output, dpc=distance, coord='18h00m00.0s +25d00m00.00s') #writting 3d fits cube
#fix header of fits file written by radmc3dpy
hdul = fits.open(output, mode='update')
hdul[0].header['CTYPE3'] = 'FREQ'
hdul[0].header['CUNIT3'] = 'Hz'
hdul[0].header['RESTFRQ'] = line_freq       #for H38                 #original #2.319935e11 # 2.3190093e+11
hdul[0].header['SPECSYS'] = 'LSRK'
hdul[0].header['ALTRVAL'] = 0.0
hdul[0].header['ALTRPIX'] = 25.0
hdul[0].header['VELREF'] =  257             # Local Standar of Rest: mean motion of material in the Milky Way in the neighborhood of the Sun
hdul.flush()   #### save the file resulting from the radiative transfer.  
hdul.close()

# Sky Model for line emission observation

In [ ]:
############ separate the line cube from the continuum cube
print ("separating the continuum data cube and the line data cube")
#print('Separatig Starting continuum_subtraction (convl_csub.py)')
hdul = fits.open(output)
############################### fin add noise ####################
data = hdul[0].data  #units of Jy/pix  #output from radmc
data_line = data.copy()
#data_cont = data.copy()
print('finish to read the data')
min_values_per_pixel = np.min(data,axis=0) #Ya prove que esto si funcione bien
print ('control log: the continuum was calculated and will be extracted')
for slide in range(data.shape[0]):
    data_line[slide,:,:] = data[slide,:,:] - min_values_per_pixel
    #data_cont[slide,:,:] = min_values_per_pixel
csub_image = 'img_rl_{}_line_H{}_{}pc.fits'.format('jet',n_inf, int(distance)) #, model_luminosity, distance)
#cont_image = 'img_rl_{}_cont_H{}_700pc_2.fits'.format('jet','38')
fits.writeto(csub_image,data_line,hdul[0].header,overwrite=True)
#fits.writeto(cont_image,data_cont,hdul[0].header,overwrite=True)	#QZ
hdul.close()

In [ ]:
print ("the sky model for the LINE OBSERVATION is ready")

In [ ]:
# create the ideal observation with the noise from the calculator. 
ideal_line_observation = False
if ideal_line_observation == True:
    # line 
    frequency = line_freq/1e9
    #here we use the ngVLA calculator to define the noise. Comment the next line and add a numerical value 
    #of the noise if you want to add the noise from the synthetic observations
    a_line,b_line,c_line = sigma_ps_fn("main",  freq=frequency, type_cal = 'line', theta=10e-3,t_int=time_int, 
                                       delta_v=channel_width_vel*1e3, verbose=True )
    noise_input_line = a_line[1]*1e-6     #Jy/beam  #original in uJy/beam
    print ("the line noise = {}".format(noise_input_line))
    new_beam = 9.6e-3 #arcsec = 2 mas  
    #a[0] = ""#frequency
    #a[1] is the simga_rms
    #a[2] is the T_rms
    #### end: Noise calculation
    
    #--------------------------------------------------------------------------------
    # line convolution and add noise
    #------------------------------------------------------------------------------
    print('\n')  
    print('Starting line cube convolution and adding noise (convl_csub.py)')
    convl_csub_image = 'img_rl_{}_line_convl_H{}_beam_{}_30kms_{}pc.fits'.format('jet',n_inf,int(new_beam*1e3),int(distance))
    hdul = fits.open(csub_image)
    npixsigma = (1/2.3548)*(new_beam/(3600*hdul[0].header['CDELT2']))
    print ("are {} pixels per beam".format(npixsigma))
    gauss_kernel = Gaussian2DKernel(npixsigma)
    pixperbeam = 1.442*(np.pi/4)*(new_beam**2)/((3600*hdul[0].header['CDELT2'])**2)
    convl_data = hdul[0].data.copy()
    convl_data_noise = hdul[0].data.copy()

    for element in range(hdul[0].data.shape[0]):
        fft_mp = lambda a: scipy.fft.fftn(a, workers=-1)  # use all available cores
        ifft_mp = lambda a: scipy.fft.ifftn(a, workers=-1)
        convl_data[element,:,:] = pixperbeam*convolve_fft(data_line[element,:,:],gauss_kernel, fftn=fft_mp, ifftn=ifft_mp)
        #add the noise after convolutio

    for element in range(hdul[0].data.shape[0]):
        convl_data_noise[element,:,:] = convl_data[element,:,:] + np.random.normal(0,scale=noise_input_line, size=(hdul[0].data.shape[1],hdul[0].data.shape[2])) #Jy/beam

    hdul[0].header['BMIN']= new_beam/3600
    hdul[0].header['BMAJ']= new_beam/3600
    hdul[0].header['BPA']= 0
    hdul[0].header['BUNIT']= 'JY/BEAM'
    fits.writeto(convl_csub_image,convl_data_noise,hdul[0].header,overwrite=True)
    hdul.close()
    print("finish")

In [ ]:
#clean memory
import gc
gc.collect()

# Continuum observation 

We need to change incl_lines = 0 in the radmc.inp file to only calculate the continuum  

# make the sky model :

In [ ]:
#-----------------------------------------
# This control the properties of the RADMC radiative transfer
#---------------------------------------
NPIX = npix = 2000             # cdelt = 0.41 mas/pix resolution ngVLA observation 2mas
SIZE_REGION = size_region = 300*2       # au

#------------------------------------------
# This control the synthetic observation sky models creation
#------------------------------------------
#BEAM = beam = (4.7e-3,4.7e-3)      # arcsec
TIME_INT = time_int = 3600*5        #1hr # second
DISTANCE = distance = 700        # pc
# RADIATIVE TRANSFER FOR A CONTINUUM IMAGES to input in CASA
########################################## RADIATIVE TRANSFER WITH RADMC-3D #################################### 
central_frequency = 92.03 * asu.GHz 
lambda_ini = (central_frequency + 10 * asu.GHz).to(asu.micron, equivalencies=asu.spectral())
lambda_fin = (central_frequency - 10 * asu.GHz).to(asu.micron, equivalencies=asu.spectral())
print (lambda_ini, lambda_fin)

########################### RRL H38##############################################  
radmc_rl = os.path.join(radmc_path, 'radmc3d')
subprocess.run(radmc_rl+' image lambdarange {} {} nlam {} incl {} npix {} sizeau {} setthreads 8 incl_lines 0'.format(lambda_ini.value,lambda_fin.value,1, 60 ,npix,size_region), shell=True, executable='/bin/bash') 
os.system('mv image.out skymodel_jet_continuum_93GHz_incl60.out')

In [ ]:
im=image.readImage('skymodel_jet_continuum_93GHz_incl60.out')
## create the fits not necesary for the ideal observations space parameters
distance=2000
output = 'img_rl_{}_{}_93GHz_{}pc.fits'.format('jet','continuum',int(distance))
im.writeFits(fname=output, dpc=distance, coord='18h00m00.0s +25d00m00.00s') #writting 3d fits cube
#fix header of fits file written by radmc3dpy
hdul = fits.open(output, mode='update')
hdul[0].header['CTYPE3'] = 'FREQ'
hdul[0].header['CUNIT3'] = 'Hz'
hdul[0].header['RESTFRQ'] = central_frequency.value #line_freq       #for H38                 #original #2.319935e11 # 2.3190093e+11
hdul[0].header['SPECSYS'] = 'LSRK'
hdul[0].header['ALTRVAL'] = 0.0
hdul[0].header['ALTRPIX'] = 25.0
hdul[0].header['VELREF'] =  257             # Local Standar of Rest: mean motion of material in the Milky Way in the neighborhood of the Sun
hdul.flush()   #### save the file resulting from the radiative transfer.  

add

In [ ]:
ideal_continuum_observation = False
if ideal_continuum_observation == True:
    #------------------------------------------------------
    #Noise calculation
    #continuum for now the noise is in the entire bandwidth 20GHz
    a_cont,b_cont,c_cont = sigma_ps_fn("main",  freq=106, type_cal = 'continuum', 
                                       theta = 5.0e-3, t_int=time_int, verbose=True )
    
    noise_input_continuum = a_cont[1]*1e-6   #Jy/beam  #original in uJy/beam
    print ("the continuum noise = {}".format(noise_input_continuum))

    ################ continuum cube + noise
    print('\n')
    print('Continuum +noise image (convl_csub.py)')

    convl_c_image = 'img_rl_{}_{}_convl_93GHz_{}pc.fits'.format('jet','continuum',int(distance))	#QZ
    hdul = fits.open(output)
    new_beam = 2.5e-3 #arcsec #arcsec
    npixsigma = (1/2.3548)*(new_beam/(3600*hdul[0].header['CDELT2']))
    gauss_kernel = Gaussian2DKernel(npixsigma,npixsigma)
    pixperbeam = 1.442*(np.pi/4)*(new_beam**2)/((3600*hdul[0].header['CDELT2'])**2)
    data_cont = hdul[0].data
    convl_data = hdul[0].data.copy()
    convl_data_noise = hdul[0].data.copy()


    for element in range(hdul[0].data.shape[0]):
        convl_data[element,:,:] = pixperbeam*convolve(data_cont[element,:,:],gauss_kernel)

    #adding noise
    for element in range(hdul[0].data.shape[0]):
        convl_data_noise[element,:,:] = convl_data[element,:,:] + np.random.normal(0,scale=noise_input_continuum, size=(hdul[0].data.shape[1],hdul[0].data.shape[2])) #Jy/beam

    hdul[0].header['BMIN']= new_beam/3600
    hdul[0].header['BMAJ']= new_beam/3600
    hdul[0].header['BPA']= 0
    hdul[0].header['BUNIT']= 'JY/BEAM'
    fits.writeto(convl_c_image,convl_data_noise,hdul[0].header,overwrite=True)
    hdul.close()

    print('\n')
    print('Finished running convolution and continuum subtraction')
    print ('Ellapsed time for run RADMC: %.3fs' % (time.time() - t0))
    print ('-------------------------------------------------\n-------------------------------------------------\n')
    ###########################################################################################################

In [ ]:
! pwd
